In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df = pd.read_csv("./Wine Data/wine_soil_data_0717.csv", index_col = "Column1" , encoding= 'cp1252')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2506 entries, 0 to 2505
Data columns (total 51 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Longitude          2506 non-null   float64
 1   Latitude           2506 non-null   float64
 2   Soil Mapping Unit  2506 non-null   int64  
 3   Continent          2506 non-null   object 
 4   Country            2506 non-null   object 
 5   Region Lv1         2489 non-null   object 
 6   Grape_Variety      2501 non-null   object 
 7   SHARE_x            2506 non-null   int64  
 8   WRB4_x             2506 non-null   object 
 9   KOPPEN             2506 non-null   object 
 10  TEXTURE_USDA_x     2430 non-null   float64
 11  REF_BULK_DENSITY   2430 non-null   float64
 12  BULK_DENSITY       2506 non-null   float64
 13  DRAINAGE_x         2430 non-null   object 
 14  ROOT_DEPTH_x       2506 non-null   int64  
 15  AWC_x              2506 non-null   int64  
 16  ROOTS_x            724 non-nu

In [20]:
df['Region Lv1'].unique()

array(['R’o Negro Province', 'NeuquŽn Province', 'Southern Chile',
       'Central Valley', 'Robertson', 'California', 'Colorado',
       'Littoral Croatia', 'Alsace', 'Bordeaux', 'Burgundy',
       'Champagne Ð Champagne', 'Languedoc-Roussillon', 'Rh™ne',
       'Alazani Valley', 'Piedmont', 'Trentino-Alto Adige', 'Veneto',
       'Tikves', 'Carcavelos', 'Lower Murray', 'Mount Lofty Ranges',
       "Hawke's Bay", 'Port Phillip', 'Buenos Aires Province',
       'Aconcagua', 'Coquimbo', 'Mendoza Province', 'Maldonado',
       'Montevideo', 'Canelones', 'San JosŽ', 'Colonia', 'Elim', 'Elgin',
       'Stellenbosch', 'Swartland', 'Franschhoek', 'Paarl', 'Tulbagh',
       'Constantia', 'Durbanville', 'Breede River Valley', 'Little Karoo',
       'San Juan Province', 'Salta Province', 'La Rioja Province',
       'Orange River Valley', 'Rio Grande do Sul', 'Paran‡',
       'Santa Catarina', 'Tarija Department', 'Ica Region', 'QuerŽtaro',
       'Coahuila / Durango', 'Zacatecas', 'central nort

In [19]:
# Region 에 (GI) 붙어있는 경우 해당 문자열 삭제

import re

def clean_text(text):
    if pd.isna(text):
        return text
    return re.sub(r'\s*\([^)]*\)', '', text).strip()

df['Region Lv1'] = df['Region Lv1'].apply(clean_text)

In [24]:
# 결측치 있는 Column만 확인
na_counts = df.isna().sum()
col_na = na_counts[na_counts != 0]
col_na

Region Lv1            17
Grape_Variety          5
TEXTURE_USDA_x        76
REF_BULK_DENSITY      76
DRAINAGE_x            76
ROOTS_x             1782
IL_x                1782
ROOTS_y             1782
IL_y                1782
SWR                 1992
DRAINAGE_y            76
TEXTURE_USDA_y        76
REF_BULK              76
dtype: int64

In [26]:
# 결측치가 있는 Column의 전체 데이터 대비 결측치 비율
col_na/len(df)*100

Region Lv1           0.678372
Grape_Variety        0.199521
TEXTURE_USDA_x       3.032721
REF_BULK_DENSITY     3.032721
DRAINAGE_x           3.032721
ROOTS_x             71.109338
IL_x                71.109338
ROOTS_y             71.109338
IL_y                71.109338
SWR                 79.489226
DRAINAGE_y           3.032721
TEXTURE_USDA_y       3.032721
REF_BULK             3.032721
dtype: float64

결측치가 70% 이상 존재하는 Column
- AWC(식물 성장을 위한 토양 최대 수분 보유량), DRAINAGE(토양 배수 능력) Column이 있어서 제외하고 가도 될듯 함

- ROOTS_x : 최상위 토양 뿌리 장애물 깊이
- IL_x : 토양 프로파일 내에 있는 불투수층
- ROOTS_y : 1번째 레이어 토양 뿌리 장애물 깊이
- IL_y : 토양 프로파일 내에 있는 불투수층
- SWR : 토양 수분 체제

In [ ]:
# Drop할 Column 
col_drop = ['ROOTS_x', 'IL_x', 'ROOTS_y', 'IL_y', 'SWR']
df.drop(col_drop, axis =1, inplace = True)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2506 entries, 0 to 2505
Data columns (total 46 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Longitude          2506 non-null   float64
 1   Latitude           2506 non-null   float64
 2   Soil Mapping Unit  2506 non-null   int64  
 3   Continent          2506 non-null   object 
 4   Country            2506 non-null   object 
 5   Region Lv1         2489 non-null   object 
 6   Grape_Variety      2501 non-null   object 
 7   SHARE_x            2506 non-null   int64  
 8   WRB4_x             2506 non-null   object 
 9   KOPPEN             2506 non-null   object 
 10  TEXTURE_USDA_x     2430 non-null   float64
 11  REF_BULK_DENSITY   2430 non-null   float64
 12  BULK_DENSITY       2506 non-null   float64
 13  DRAINAGE_x         2430 non-null   object 
 14  ROOT_DEPTH_x       2506 non-null   int64  
 15  AWC_x              2506 non-null   int64  
 16  HWSD2_SMU_ID_y     2506 non-n

In [33]:
# 76개의 같은 인덱스가 아래 6개 Column에 대해 결측치를 가지고 있는 것으로 확인
df[df['TEXTURE_USDA_x'].isna()][['TEXTURE_USDA_x', 'REF_BULK_DENSITY', 'DRAINAGE_x', 'DRAINAGE_y', 'TEXTURE_USDA_y', 'REF_BULK']].isna().sum()

TEXTURE_USDA_x      76
REF_BULK_DENSITY    76
DRAINAGE_x          76
DRAINAGE_y          76
TEXTURE_USDA_y      76
REF_BULK            76
dtype: int64

In [37]:
# 결측치 76개중 75개는 TC로 농업이 불가능한 토양으로 삭제 / 전체 TC 데이터가 모두 여기에 포함되어 있음
# WR 또한 전체 데이터 중 1개로 해당 row 삭제
df[df['TEXTURE_USDA_x'].isna()]['WRB4_x'].value_counts()

WRB4_x
TC    75
WR     1
Name: count, dtype: int64

In [36]:
df['WRB4_x'].value_counts()

WRB4_x
CMeu         309
PLeu         268
LVcr         214
LVha         121
CMca         110
LPli          93
CMdy          92
CLpt          87
TC            75
ACha          65
LXfr          64
PHlv          60
PHha          59
LVcc          55
LVgl          53
RGeu          50
UMcm          49
LPeu          48
RGca          47
CLha          43
LPrz          38
PZal          36
CMcr          34
VR            34
KSha          34
PZcb          33
ACgl          33
SNha          23
GLeu          22
CMvr          22
FLca          22
FLeu          21
AR            20
VRha          18
CHha          18
NT            17
Nthu          14
UMac          12
ALha          11
RGdy          11
FRro          10
CHcc           7
HS             6
FLdy           6
PLdy           5
STrt           4
PHcc           4
LVab           4
ANvi           4
PHgl           3
LVvr           3
SCha           3
LPmo           3
ATpa&ATtr      2
RTeu           2
ARcq           1
KSlv           1
GLum           1
WR     

In [42]:
drop_row = df[df['TEXTURE_USDA_x'].isna()].index.to_list()
len(drop_row)

76

In [43]:
df.drop(drop_row, axis = 0, inplace = True)
df.isna().sum()

Longitude             0
Latitude              0
Soil Mapping Unit     0
Continent             0
Country               0
Region Lv1           17
Grape_Variety         5
SHARE_x               0
WRB4_x                0
KOPPEN                0
TEXTURE_USDA_x        0
REF_BULK_DENSITY      0
BULK_DENSITY          0
DRAINAGE_x            0
ROOT_DEPTH_x          0
AWC_x                 0
HWSD2_SMU_ID_y        0
SEQUENCE              0
SHARE_y               0
WRB4_y                0
ROOT_DEPTH_y          0
DRAINAGE_y            0
AWC_y                 0
COARSE                0
SAND                  0
SILT                  0
CLAY                  0
TEXTURE_USDA_y        0
TEXTURE_SOTER         0
BULK                  0
REF_BULK              0
ORG_CARBON            0
PH_WATER              0
TOTAL_N               0
CN_RATIO              0
CEC_SOIL              0
CEC_CLAY              0
CEC_EFF               0
TEB                   0
BSAT                  0
ALUM_SAT              0
ESP             

In [44]:
na_counts = df.isna().sum()
col_na = na_counts[na_counts != 0]
col_na

Region Lv1       17
Grape_Variety     5
dtype: int64

In [45]:
# Region Lv1의 누락 값은 머신러닝에 영향을 주지 않는 요소로 그대로 둠
[df['Region Lv1'].isna()]

,Longitude,Latitude,Soil Mapping Unit,Continent,Country,Region Lv1,Grape_Variety,SHARE_x,WRB4_x,KOPPEN,...,CEC_CLAY,CEC_EFF,TEB,BSAT,ALUM_SAT,ESP,TCARBON_EQ,GYPSUM,ELEC_COND,Top10_Grape
Column1,,,,,,,,,,,,,,,,,,,,,
920,18.325000,42.708333,10530,Europe,Bosnia and Herzegovina,NaN,vranac,60,LPli,C,...,35,20,10,64,0,3,0.0,0.3,1,0
921,18.333333,42.708333,10530,Europe,Bosnia and Herzegovina,NaN,vranac,60,LPli,C,...,35,20,10,64,0,3,0.0,0.3,1,0
925,17.541667,43.208333,10530,Europe,Bosnia and Herzegovina,NaN,blatina,60,LPli,C,...,35,20,10,64,0,3,0.0,0.3,1,0
926,17.691667,43.225000,10530,Europe,Bosnia and Herzegovina,NaN,blatina,60,LPli,C,...,35,20,10,64,0,3,0.0,0.3,1,0
927,17.691667,43.225000,10530,Europe,Bosnia and Herzegovina,NaN,vranac,60,LPli,C,...,35,20,10,64,0,3,0.0,0.3,1,0
928,17.691667,43.225000,10530,Europe,Bosnia and Herzegovina,NaN,plavac_mali,60,LPli,C,...,35,20,10,64,0,3,0.0,0.3,1,0
935,17.691667,43.216667,10523,Europe,Bosnia and Herzegovina,NaN,blatina,75,FLca,D,...,55,30,30,99,0,4,22.6,0.5,2,0
936,17.691667,43.216667,10523,Europe,Bosnia and Herzegovina,NaN,vranac,75,FLca,D,...,55,30,30,99,0,4,22.6,0.5,2,0
937,17.691667,43.216667,10523,Europe,Bosnia and Herzegovina,NaN,plavac_mali,75,FLca,D,...,55,30,30,99,0,4,22.6,0.5,2,0


In [46]:
# 이전 데이터 확인해보니 데이터 클렌징하면서 잘못된 값 삭제하며 Null값으로 변환함
# Top10 Grape 에서 1 or 0으로 나뉘니 여기도 두고 가도 좋을 듯 함.. 포도를 재배하는 토양 특성을 살리기 위해?
df[df['Grape_Variety'].isna()]

,Longitude,Latitude,Soil Mapping Unit,Continent,Country,Region Lv1,Grape_Variety,SHARE_x,WRB4_x,KOPPEN,...,CEC_CLAY,CEC_EFF,TEB,BSAT,ALUM_SAT,ESP,TCARBON_EQ,GYPSUM,ELEC_COND,Top10_Grape
Column1,,,,,,,,,,,,,,,,,,,,,
97,-72.550000,-36.583333,13468,Americas,Chile,Southern Chile,NaN,50,ALha,C,...,26,6,5,47,21,1,0.0,0.1,0,0
98,-72.675000,-36.583333,13468,Americas,Chile,Southern Chile,NaN,50,ALha,C,...,26,6,5,47,21,1,0.0,0.1,0,0
99,-72.666667,-36.583333,13468,Americas,Chile,Southern Chile,NaN,50,ALha,C,...,26,6,5,47,21,1,0.0,0.1,0,0
706,-116.591667,32.500000,18265,Americas,Mexico,Baja California,NaN,43,LPli,B,...,35,25,10,64,0,3,6.3,0.3,1,0
1255,4.850000,44.108333,9400,Europe,France,Rh™ne,NaN,90,FLca,C,...,52,29,28,100,0,2,10.1,0.5,2,0


In [48]:
# 중간 저장하기
df.to_csv("./Wine Data/wine_soil_EDA_v1.csv", index = True)

### 데이터 시각화

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2430 entries, 0 to 2505
Data columns (total 46 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Longitude          2430 non-null   float64
 1   Latitude           2430 non-null   float64
 2   Soil Mapping Unit  2430 non-null   int64  
 3   Continent          2430 non-null   object 
 4   Country            2430 non-null   object 
 5   Region Lv1         2413 non-null   object 
 6   Grape_Variety      2425 non-null   object 
 7   SHARE_x            2430 non-null   int64  
 8   WRB4_x             2430 non-null   object 
 9   KOPPEN             2430 non-null   object 
 10  TEXTURE_USDA_x     2430 non-null   float64
 11  REF_BULK_DENSITY   2430 non-null   float64
 12  BULK_DENSITY       2430 non-null   float64
 13  DRAINAGE_x         2430 non-null   object 
 14  ROOT_DEPTH_x       2430 non-null   int64  
 15  AWC_x              2430 non-null   int64  
 16  HWSD2_SMU_ID_y     2430 non-n

In [51]:
# 토양 관련 Column만 통계 수치 살펴보기
df[df.columns[7:]].describe()

,SHARE_x,TEXTURE_USDA_x,REF_BULK_DENSITY,BULK_DENSITY,ROOT_DEPTH_x,AWC_x,HWSD2_SMU_ID_y,SEQUENCE,SHARE_y,ROOT_DEPTH_y,...,CEC_CLAY,CEC_EFF,TEB,BSAT,ALUM_SAT,ESP,TCARBON_EQ,GYPSUM,ELEC_COND,Top10_Grape
count,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.0,2430.000000,2430.000000,...,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000
mean,56.274897,8.310700,1.724984,1.381424,1.443210,130.379424,10815.129630,1.0,56.274897,1.443210,...,47.946914,17.228395,16.152263,76.261728,4.522634,2.252675,3.109506,0.682016,0.897119,0.750617
std,17.169012,2.265464,0.145701,0.136292,0.892691,54.090512,7297.584455,0.0,17.169012,0.892691,...,15.269962,10.355164,10.577590,19.520556,9.370870,2.279995,6.749505,0.925523,0.826755,0.432745
min,20.000000,3.000000,0.970000,0.540000,1.000000,10.000000,3334.000000,1.0,20.000000,1.000000,...,9.000000,3.000000,2.000000,10.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,45.000000,7.000000,1.700000,1.300000,1.000000,110.000000,6006.000000,1.0,45.000000,1.000000,...,38.000000,9.000000,8.000000,71.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
50%,50.000000,9.000000,1.720000,1.410000,1.000000,150.000000,9658.000000,1.0,50.000000,1.000000,...,47.000000,15.000000,14.000000,80.000000,0.000000,2.000000,0.000000,0.100000,1.000000,1.000000
75%,70.000000,9.000000,1.810000,1.450000,1.000000,170.000000,12242.000000,1.0,70.000000,1.000000,...,57.000000,21.000000,20.000000,93.000000,0.000000,3.000000,1.600000,1.600000,1.000000,1.000000
max,100.000000,12.000000,2.030000,1.720000,4.000000,214.000000,30414.000000,1.0,100.000000,4.000000,...,110.000000,52.000000,50.000000,100.000000,40.000000,38.000000,27.300000,4.000000,16.000000,1.000000
